In [4]:
import getGmail_Class
import os, sys, logging
from tqdm import trange
from datetime import datetime
import pandas as pd

In [2]:
gGC = getGmail_Class.Email()

In [6]:
Num, Name, investment_company, Date, Path = [[] for i in range(5)]

# Create log file
if not os.path.isdir("log"):
    os.mkdir("log")

FORMAT = '%(asctime)s %(levelname)s: %(message)s'
logging.basicConfig(level = logging.INFO, filename = "./log/" + datetime.now().strftime("%Y_%m_%d") + '.log', filemode = 'w', format = FORMAT)
logging.info('Updating email start')

# request a list of all the messages
result = gGC.service.users().messages().list(userId = 'me', maxResults = 500, labelIds = ["INBOX"]).execute()
messages = result.get('messages')

if len(messages) == 0:
    logging.info('Inbox quantity is 0')
    logging.info('Updating email end')
    sys.exit(0)

# iterate through all the messages
for i in trange(len(messages)):
    # Get the message from its id
    txt = gGC.service.users().messages().get(userId = 'me', id = messages[i]['id']).execute()
    payload = txt['payload']
    headers = payload['headers']
    date = gGC.getDate(headers)

    if gGC.verifySubject(headers):
        for d in headers:
            if d['name'] == 'Subject':
                subject = d['value']

        tempNum, tempName, tempInvestment_company, tempPath = gGC.getResearch_report(subject, payload, messages[i]['id'])

        if len(tempNum) != 0:
            Num += tempNum
            Name += tempName
            Date.append(date)
            investment_company.append(tempInvestment_company)
            Path += tempPath
#                     # Modify labels
#                     self.modifyLabels(messages[i]['id'], "Label2")
    else:
        # Modify labels
        gGC.modifyLabels(messages[i]['id'], "Label3")

df = pd.DataFrame({ "Number" : Num, "Name" : Name, "Investment company" : investment_company, "Date" : Date, "File path" : Path })

logging.info('Updating email end')

100%|█████████████████████████████████████████| 500/500 [19:56<00:00,  2.39s/it]


In [7]:
df

,Number,Name,Investment company,Date,File path
0,3563,牧德,元富,2020_8_17,./file/3563/3563牧德20200817-報告.pdf
1,6251,定穎,,2020_8_17,./file/6251/20200817-個股報告-定穎(6251).pdf
2,3665,貿聯-KY,,2020_8_17,./file/3665/20200817-個股報告-貿聯-KY(3665).pdf
3,1907,永豐餘,元富,2020_8_17,./file/1907/永豐餘(1907)-20200817-速報.pdf
4,1905,華紙,元富,2020_8_17,./file/1905/華紙(1905)-20200817-速報.pdf
...,...,...,...,...,...
171,5215,科嘉-KY,,2020_7_24,./file/5215/20200724-個股報告-科嘉(5215).pdf
172,6504,南六,元富,2020_7_24,./file/6504/南六(6504)-20200724-速報.pdf
173,6116,彩晶,第一金,2020_7_23,./file/6116/彩晶(6116)-區間操作.pdf
174,6116,彩晶,第一金,2020_7_23,./file/6116/彩晶(6116)-區間操作.pdf
